### Import modules

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import psycopg2 as db

### Get data from csv file

In [3]:
users_df = pd.read_csv("500-us-users.csv")
users_df.tail()

,first_name,last_name,address,city,county,state,zip,phone1,phone2
495,Brittni,Gillaspie,67 Rv Cent,Boise,Ada,ID,83709,208-709-1235,208-206-9848
496,Raylene,Kampa,2 Sw Nyberg Rd,Elkhart,Elkhart,IN,46514,574-499-1454,574-330-1884
497,Flo,Bookamer,89992 E 15th St,Alliance,Box Butte,NE,69301,308-726-2182,308-250-6987
498,Jani,Biddy,61556 W 20th Ave,Seattle,King,WA,98104,206-711-6498,206-395-6284
499,Chauncey,Motley,63 E Aurora Dr,Orlando,Orange,FL,32804,407-413-4842,407-557-8857


### Connecting to DB

In [2]:
connection = db.connect(database="postgres", user="postgres", password="postgres", host="localhost", port="5432")

### Creating Schema

In [18]:
cursor = connection.cursor()
cursor.execute("CREATE SCHEMA IF NOT EXISTS people")
connection.commit()
cursor.close()

### Create Tables

In [19]:
cursor = connection.cursor()
cursor.execute("""CREATE TABLE IF NOT EXISTS people.users(
    user_id serial PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    address VARCHAR(255) NOT NULL,
    city VARCHAR(100) NOT NULL,
    county VARCHAR(100) NOT NULL,
    state VARCHAR(3) NOT NULL,
    zip VARCHAR(15) NOT NULL
)
""")
cursor.execute("""CREATE TABLE IF NOT EXISTS people.phone_numbers(
    phone_number_id serial NOT NULL,
    user_id INT NOT NULL,
    phone_number VARCHAR(12) NOT NULL,
    FOREIGN KEY (user_id) REFERENCES people.users (user_id)
)
""")
connection.commit()
cursor.close()

### Create INDEX on ZIP in USERS table

In [20]:
cursor = connection.cursor()
cursor.execute("CREATE INDEX IF NOT EXISTS zip_index ON people.users (zip)")
connection.commit()
cursor.close()

### INSERT data into tables

In [24]:
cursor = connection.cursor()
# cursor.execute("ROLLBACK")
# truncate table before running
# cursor.execute("TRUNCATE ONLY people.users, people.phone_numbers RESTART IDENTITY")
for index, row in users_df.iterrows():
    cursor.execute("""
        INSERT INTO people.users (first_name,last_name,address,city,county,state,zip)
        VALUES (%s,%s,%s,%s,%s,%s,%s) RETURNING user_id
        """,
        (row["first_name"],row["last_name"],row["address"],row["city"],row["county"],row["state"],row["zip"])
    )
    last_id = cursor.fetchone()[0]
    cursor.execute("""
        INSERT INTO people.phone_numbers (user_id,phone_number)
        VALUES (%s,%s)
        """,
        (last_id,row["phone1"])
    )
    cursor.execute("""
        INSERT INTO people.phone_numbers (user_id,phone_number)
        VALUES (%s,%s)
        """,
        (last_id,row["phone2"])
    )
connection.commit()
cursor.close()

### How many people are from each state ??

In [3]:
cursor = connection.cursor()
cursor.execute("""
    SELECT u.state, COUNT(*) AS "No. of people"
    FROM people.users AS u
    GROUP BY u.state
""")
result = cursor.fetchall()
cursor.close()
print(result)

[('CA', 72), ('NH', 1), ('OR', 6), ('ND', 1), ('TX', 32), ('NV', 2), ('KY', 1), ('OH', 22), ('NY', 46), ('HI', 4), ('NM', 2), ('MS', 4), ('IN', 9), ('DC', 1), ('NE', 2), ('MO', 4), ('FL', 28), ('ME', 3), ('AR', 1), ('CT', 5), ('NC', 8), ('WI', 11), ('SD', 1), ('OK', 1), ('RI', 5), ('GA', 7), ('ID', 4), ('MN', 7), ('PA', 29), ('MD', 17), ('AK', 6), ('WY', 3), ('LA', 9), ('MT', 1), ('IL', 15), ('TN', 10), ('WA', 8), ('NJ', 52), ('MI', 14), ('MA', 12), ('UT', 1), ('IA', 1), ('CO', 8), ('SC', 3), ('VA', 7), ('AZ', 9), ('KS', 5)]


In [4]:
cursor = connection.cursor()
cursor.execute("""
    SELECT 
        (u.first_name || ' ' || u.last_name) AS "Name",
        u.zip
    FROM people.users AS u
    WHERE u.zip in (
        SELECT
            u.zip AS "zip_code"
        FROM people.users AS u
        GROUP BY u.zip
        HAVING COUNT(*) > 1
    )
    ORDER BY u.zip
""")
result = cursor.fetchall()
cursor.close()
print(result)

[('Jose Stockham', '10011'), ('Layla Springe', '10011'), ('Willow Kusko', '10011'), ('Cyril Daufeldt', '10013'), ('Derick Dhamer', '10013'), ('Haydee Denooyer', '10016'), ('Jess Chaffins', '10016'), ('Gregoria Pawlowicz', '11530'), ('Gilma Liukko', '11530'), ('Donte Kines', '1602'), ('Lucy Treston', '1602'), ('Edna Miceli', '16502'), ('Elly Morocco', '16502'), ('Franklyn Emard', '19103'), ('Ronny Caiafa', '19103'), ('Lisha Centini', '22102'), ('Lavonna Wolny', '22102'), ('Salome Lacovara', '23219'), ('Jerry Dallen', '23219'), ('Jade Farrar', '29201'), ('Sabra Uyetake', '29201'), ('Renea Monterrubio', '30328'), ('Sarah Candlish', '30328'), ('Caprice Suell', '37211'), ('Audra Kohnert', '37211'), ('Casie Good', '37211'), ('Lorrie Nestle', '37388'), ('Jesusa Shin', '37388'), ('Bernardine Rodefer', '38112'), ('Linn Paa', '38112'), ('Goldie Schirpke', '4401'), ('Jolanda Hanafan', '4401'), ('Adelina Nabours', '44103'), ('Kattie Vonasek', '44103'), ('Cyndy Goldammer', '55337'), ('Quentin Birkn

In [5]:
cursor = connection.cursor()
cursor.execute("""
    SELECT
        LEFT(u.last_name,1) AS "First letter",
        COUNT(*)
    FROM people.users AS u
    GROUP BY LEFT(u.last_name,1)
""")
result = cursor.fetchall()
cursor.close()
print(result)

[('N', 10), ('O', 7), ('V', 10), ('L', 27), ('Z', 4), ('M', 46), ('D', 23), ('G', 22), ('J', 8), ('P', 26), ('I', 4), ('U', 2), ('K', 30), ('A', 19), ('Y', 4), ('R', 25), ('W', 20), ('E', 9), ('B', 41), ('C', 35), ('T', 18), ('H', 29), ('F', 22), ('S', 59)]
